# MIEX Cu–Mo Prospectivity (Notebook)

This notebook is a **clean, function-based refactor** of the original analysis notebook.
It keeps the same workflow, but organizes it into reusable functions and a single
`run_pipeline()` entry point.

**Main outputs**
- Site table with geology + distance features
- Knowledge-derived embedding priors (deposit-model similarity)
- Spatial-CV anomaly/prospectivity models (baseline vs. knowledge-augmented)
- Optional map products (interpolated prospectivity surface, target polygons)

> **Note:** Secrets (Azure/OpenAI) and local file paths are **not committed**. Configure via environment variables or the config cell below.

## 0) Setup & Configuration
Fill in paths and options.  
If you run in Colab, you may still mount Drive manually, but the notebook no longer *requires* Colab-specific APIs.

In [ ]:
from __future__ import annotations

import os, json, glob, hashlib, math, warnings
from dataclasses import dataclass
from typing import Dict, List, Optional, Tuple

import numpy as np
import pandas as pd

warnings.filterwarnings("ignore")

# Geo / mapping stack (install if needed)
import geopandas as gpd
from shapely.geometry import Point
from shapely.ops import unary_union
from shapely.strtree import STRtree

from pyproj import CRS, Transformer

# ML
from sklearn.model_selection import GroupKFold
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import RobustScaler
from sklearn.impute import SimpleImputer
from sklearn.decomposition import PCA
from sklearn.metrics import roc_auc_score, average_precision_score
from sklearn.ensemble import HistGradientBoostingClassifier

# -----------------------
# User config (edit me)
# -----------------------
CONFIG = {
    # Inputs
    "csv_points": "data/Cumo.csv",             # <-- path to point CSV
    "geology_root": "data/geology/",           # <-- folder containing geology shapefiles
    "json_dir": "data/deposit_models_json/",   # <-- folder containing 87 deposit-model JSONs

    # Column hints (set if auto-detect fails)
    "lon_candidates": ["Longitude", "lon", "LONGITUDE", "X", "LONG"],
    "lat_candidates": ["Latitude", "lat", "LATITUDE", "Y", "LAT"],

    # Spatial settings
    "metric_crs": "EPSG:32611",  # UTM zone used in the original notebook
    "crop_frac": 0.10,           # keep ~10% window for quick testing; set 1.0 for full run
    "site_grid_m": 5.0,          # merge duplicate points within this grid (meters)
    "distance_cap_m": 25_000.0,  # cap distances for stability

    # Knowledge priors (embeddings)
    "embedding_provider": "azure_openai",      # "azure_openai" | "openai"
    "embedding_model": os.getenv("EMBEDDING_MODEL", "text-embedding-3-large"),
    "embedding_cache": "cache/embeddings/",

    # Modeling
    "n_splits": 5,
    "top_frac_eval": 0.10,  # recall@topX
}
os.makedirs(CONFIG["embedding_cache"], exist_ok=True)

## 1) Utilities (I/O, CRS, helpers)

In [ ]:
def read_csv_flex(path: str) -> pd.DataFrame:
    """Read CSV with a Latin-1 fallback for messy encodings."""
    try:
        return pd.read_csv(path, low_memory=False)
    except UnicodeDecodeError:
        return pd.read_csv(path, encoding="latin1", low_memory=False)

def pick_col(cols: List[str], candidates: List[str]) -> Optional[str]:
    """Pick a column name from candidates with tolerant matching."""
    lower = {c.lower(): c for c in cols}
    for c in candidates:
        if c.lower() in lower:
            return lower[c.lower()]
    # fuzzy contains
    for col in cols:
        col_norm = col.lower().replace(" ", "")
        for c in candidates:
            if c.lower().replace(" ", "") in col_norm:
                return col
    return None

def to_metric_points(df: pd.DataFrame, lon_col: str, lat_col: str, metric_crs: str) -> gpd.GeoDataFrame:
    """Create GeoDataFrame from lon/lat and project to metric CRS."""
    gdf = gpd.GeoDataFrame(
        df.copy(),
        geometry=gpd.points_from_xy(df[lon_col].astype(float), df[lat_col].astype(float)),
        crs="EPSG:4326"
    )
    return gdf.to_crs(metric_crs)

def crop_window(gdf: gpd.GeoDataFrame, frac: float) -> gpd.GeoDataFrame:
    """Keep a center crop window for faster iteration. Use frac=1.0 for full."""
    if frac is None or frac >= 0.999:
        return gdf
    xmin, ymin, xmax, ymax = gdf.total_bounds
    cx, cy = (xmin+xmax)/2, (ymin+ymax)/2
    w, h = (xmax-xmin)*frac, (ymax-ymin)*frac
    box = gpd.GeoSeries([Point(cx, cy).buffer(1).envelope], crs=gdf.crs).iloc[0]
    # make a rectangle envelope
    rect = gpd.GeoSeries([Point(cx, cy).buffer(1).envelope], crs=gdf.crs).iloc[0]
    rect = gpd.GeoSeries([Point(cx, cy).buffer(1).envelope], crs=gdf.crs).iloc[0]
    # build manual rectangle
    from shapely.geometry import box as sbox
    rect = sbox(cx - w/2, cy - h/2, cx + w/2, cy + h/2)
    return gdf[gdf.geometry.within(rect)].copy()

def round_series(x: np.ndarray, step: float) -> np.ndarray:
    """Round to a metric grid (used to merge near-duplicate samples)."""
    x = np.asarray(x, dtype=float)
    return (np.round(x/step)*step).astype(np.float64)

def safe_min_distance(points: np.ndarray, lines_gdf: gpd.GeoDataFrame, cap_m: float) -> np.ndarray:
    """Compute min distance from each point to a line layer using STRtree. Returns capped meters."""
    if lines_gdf is None or len(lines_gdf) == 0:
        return np.full(len(points), np.nan, dtype=float)

    geoms = list(lines_gdf.geometry.values)
    tree = STRtree(geoms)

    out = np.empty(len(points), dtype=float)
    for i, p in enumerate(points):
        # query candidates by bbox; STRtree returns geometries
        cands = tree.query(p)
        if not cands:
            out[i] = np.nan
            continue
        out[i] = float(min(p.distance(g) for g in cands))
    if cap_m is not None:
        out = np.minimum(out, cap_m)
    return out

## 2) Load geology layers (polygons + line features)
The original notebook searched the geology folder for:
- a **polygon** layer (map units)
- optional **line** layers (faults / contacts / boundaries)

The helper below keeps the same behavior with clear fallbacks.

In [ ]:
def find_polygon_shp(root: str) -> str:
    """Find a likely map-unit polygon shapefile under a root folder."""
    cand = []
    for r, _, fs in os.walk(root):
        for f in fs:
            if f.lower().endswith('.shp'):
                fl = f.lower()
                fp = os.path.join(r, f)
                if 'mapunit' in fl and ('poly' in fl or 'polygon' in fl):
                    cand.append(fp)
    if cand:
        return cand[0]
    # fallback: first .shp we can read as polygons
    for r, _, fs in os.walk(root):
        for f in fs:
            if f.lower().endswith('.shp'):
                return os.path.join(r, f)
    raise FileNotFoundError(f"No shapefile found under {root}")

def load_geology_layers(geology_root: str, metric_crs: str) -> Tuple[gpd.GeoDataFrame, Dict[str, gpd.GeoDataFrame]]:
    """Load map-unit polygons and candidate line layers projected to metric CRS."""
    poly_path = find_polygon_shp(geology_root)
    polys = gpd.read_file(poly_path)
    polys = polys.to_crs(metric_crs)

    # load all line-ish shapefiles as candidates
    line_layers: Dict[str, gpd.GeoDataFrame] = {}
    for shp in glob.glob(os.path.join(geology_root, '**', '*.shp'), recursive=True):
        if shp == poly_path:
            continue
        try:
            g = gpd.read_file(shp)
            if g.empty:
                continue
            geom_type = g.geometry.iloc[0].geom_type.lower()
            if 'line' in geom_type:
                key = os.path.splitext(os.path.basename(shp))[0]
                line_layers[key] = g.to_crs(metric_crs)
        except Exception:
            continue
    return polys, line_layers

def pick_unit_key(cols: List[str]) -> Optional[str]:
    """Pick a unit/mapunit label field from polygon attributes."""
    L = {c.lower(): c for c in cols}
    for k in ['mapunit','unit','unitsym','symbol','label','name','unitcode','unit_id']:
        if k in L:
            return L[k]
    return None

## 3) Load points, project, and build *sites*
The original notebook merges near-duplicate samples into robust "sites" using a metric grid.

In [ ]:
def load_points_build_sites(config: dict) -> Tuple[gpd.GeoDataFrame, gpd.GeoDataFrame]:
    """Load point CSV -> metric GeoDataFrame -> crop -> aggregate into site table."""
    df = read_csv_flex(config['csv_points'])
    lon_col = pick_col(list(df.columns), config['lon_candidates'])
    lat_col = pick_col(list(df.columns), config['lat_candidates'])
    if lon_col is None or lat_col is None:
        raise RuntimeError(f"Cannot detect lon/lat columns. Found lon={lon_col}, lat={lat_col}.")

    gpts = to_metric_points(df, lon_col, lat_col, config['metric_crs'])
    gpts = crop_window(gpts, config['crop_frac'])

    # Create bins for aggregation
    G = gpts.copy()
    G['x_m'] = G.geometry.x.astype('float64')
    G['y_m'] = G.geometry.y.astype('float64')
    G['x_bin'] = round_series(G['x_m'].values, config['site_grid_m'])
    G['y_bin'] = round_series(G['y_m'].values, config['site_grid_m'])
    G['site_id'] = (G['x_bin'].astype(str) + '_' + G['y_bin'].astype(str))

    # Aggregate: keep median for numeric, first for non-numeric (same spirit as original)
    num_cols = [c for c in G.columns if pd.api.types.is_numeric_dtype(G[c]) and c not in ['x_m','y_m']]
    keep_cols = [c for c in G.columns if c not in ['geometry']]
    agg = {}
    for c in keep_cols:
        if c in num_cols:
            agg[c] = 'median'
        else:
            agg[c] = 'first'

    U = (G.drop(columns=['geometry'])
           .groupby('site_id', dropna=False)
           .agg(agg)
           .reset_index())

    # rebuild geometry from binned coordinates
    U_gdf = gpd.GeoDataFrame(
        U,
        geometry=gpd.points_from_xy(U['x_bin'].astype(float), U['y_bin'].astype(float)),
        crs=config['metric_crs']
    )
    return gpts, U_gdf

## 4) Spatial joins & distance features
- Join each site to a map-unit polygon (`unit_code`)
- Compute min distance to each line layer (fault/contact/etc.)

In [ ]:
def add_geology_unit(U: gpd.GeoDataFrame, polys: gpd.GeoDataFrame) -> gpd.GeoDataFrame:
    """Spatial join sites to polygons and create a single 'unit_code' column."""
    unit_col = pick_unit_key(list(polys.columns))
    if unit_col is None:
        raise RuntimeError('Cannot find a unit field in polygons (mapunit/unit/label/etc.).')

    out = gpd.sjoin(U, polys[[unit_col, 'geometry']], how='left', predicate='within')
    out = out.rename(columns={unit_col: 'unit_code'}).drop(columns=['index_right'], errors='ignore')
    # de-duplicate unit_code if created multiple times
    ucols = [c for c in out.columns if c.lower() == 'unit_code']
    if len(ucols) > 1:
        tmp = out[ucols].copy()
        out['unit_code'] = tmp.bfill(axis=1).iloc[:, 0]
        out = out.drop(columns=[c for c in ucols if c != 'unit_code'])
    return out

def add_line_distances(U: gpd.GeoDataFrame, line_layers: Dict[str, gpd.GeoDataFrame], cap_m: float) -> gpd.GeoDataFrame:
    """Add min-distance-to-line features for each line layer."""
    out = U.copy()
    pts = np.array(list(out.geometry.values))
    for name, g in line_layers.items():
        out[f'dist_to_{name}'] = safe_min_distance(pts, g, cap_m=cap_m)
    return out

## 5) Knowledge priors (deposit-model embeddings + sample similarity)

This reproduces the original idea:
1) Convert each deposit-model JSON into a stable text
2) Embed model texts
3) Embed per-sample text (from selected descriptive columns) and compute similarity priors

**Important:** This notebook includes a provider-agnostic embedding wrapper.  
You must set credentials via environment variables and **never commit secrets**.

In [ ]:
# -------- Embedding client wrappers --------
def _hash_text(s: str) -> str:
    return hashlib.md5(s.encode('utf-8')).hexdigest()[:12]

def build_model_text(j: dict) -> str:
    """Convert a deposit-model JSON dict into a compact, stable text for embedding."""
    keep_keys = [
        'Model_Index','Model_Name','APPROXIMATE_SYNONYM','DESCRIPTION',
        'Rock_Types','Depositional_Environment','Tectonic_Settings',
        'Alteration','Mineralogy','Ore_Controls','Geochemical_Signature',
        'Associated_Deposit_Types','Weathering','Texture_Structure','Textures',
        'Age_Range','GEOLOGIC_AGE','HOST_ROCK_TYPE','MINERALIZATION'
    ]
    lines = []
    for k in keep_keys:
        if k in j and j[k] not in [None, '', [], {}]:
            v = j[k]
            if isinstance(v, (list, tuple)):
                v = '; '.join([str(x) for x in v if x is not None])
            elif isinstance(v, dict):
                v = json.dumps(v, ensure_ascii=False)
            lines.append(f"{k}: {v}")
    return '\n'.join(lines)

def load_deposit_models(json_dir: str) -> pd.DataFrame:
    """Load deposit-model JSON files and return a table with text for embedding."""
    paths = sorted(glob.glob(os.path.join(json_dir, '*.json')))
    if not paths:
        raise FileNotFoundError(f"No JSON files found in {json_dir}")
    rows = []
    for p in paths:
        with open(p, 'r', encoding='utf-8') as f:
            j = json.load(f)
        rows.append({
            'path': p,
            'model_id': os.path.splitext(os.path.basename(p))[0],
            'model_text': build_model_text(j),
        })
    return pd.DataFrame(rows)

def get_embedding_client(config: dict):
    """Return a callable(texts)->np.ndarray. Supports Azure OpenAI or OpenAI."""
    provider = config.get('embedding_provider', 'azure_openai').lower()

    if provider == 'azure_openai':
        # Expected env vars:
        # AZURE_OPENAI_API_KEY, AZURE_OPENAI_ENDPOINT, AZURE_OPENAI_API_VERSION (optional)
        from openai import AzureOpenAI
        api_key = os.getenv('AZURE_OPENAI_API_KEY')
        endpoint = os.getenv('AZURE_OPENAI_ENDPOINT')
        api_version = os.getenv('AZURE_OPENAI_API_VERSION', '2024-02-15-preview')
        deployment = os.getenv('AZURE_OPENAI_EMBED_DEPLOYMENT', config['embedding_model'])
        if not api_key or not endpoint:
            raise RuntimeError('Missing AZURE_OPENAI_API_KEY / AZURE_OPENAI_ENDPOINT.')
        client = AzureOpenAI(api_key=api_key, azure_endpoint=endpoint, api_version=api_version)

        def embed(texts: List[str]) -> np.ndarray:
            out = []
            for t in texts:
                resp = client.embeddings.create(model=deployment, input=t)
                out.append(resp.data[0].embedding)
            return np.asarray(out, dtype=float)
        return embed

    elif provider == 'openai':
        from openai import OpenAI
        api_key = os.getenv('OPENAI_API_KEY')
        if not api_key:
            raise RuntimeError('Missing OPENAI_API_KEY.')
        client = OpenAI(api_key=api_key)
        model = config['embedding_model']

        def embed(texts: List[str]) -> np.ndarray:
            out = []
            for t in texts:
                resp = client.embeddings.create(model=model, input=t)
                out.append(resp.data[0].embedding)
            return np.asarray(out, dtype=float)
        return embed

    else:
        raise ValueError(f"Unknown embedding_provider: {provider}")

def embed_with_cache(texts: List[str], embed_fn, cache_dir: str, prefix: str) -> np.ndarray:
    """Embed texts with per-text caching to avoid re-calling the API."""
    embs = []
    for t in texts:
        hid = _hash_text(t)
        fp = os.path.join(cache_dir, f"{prefix}_{hid}.npy")
        if os.path.exists(fp):
            embs.append(np.load(fp))
        else:
            v = embed_fn([t])[0]
            np.save(fp, np.asarray(v, dtype=float))
            embs.append(v)
    return np.asarray(embs, dtype=float)

def build_sample_text(row: pd.Series, text_cols: List[str]) -> str:
    """Build a per-sample text from selected descriptive columns."""
    parts = []
    for c in text_cols:
        if c in row.index and pd.notnull(row[c]) and str(row[c]).strip() != '':
            parts.append(f"{c}: {str(row[c]).strip()}")
    return '\n'.join(parts)

def cosine_sim_matrix(A: np.ndarray, B: np.ndarray) -> np.ndarray:
    """Cosine similarity between rows of A and rows of B."""
    A = A / (np.linalg.norm(A, axis=1, keepdims=True) + 1e-12)
    B = B / (np.linalg.norm(B, axis=1, keepdims=True) + 1e-12)
    return A @ B.T

# Text columns used in the original notebook's 'deployable' set (safe defaults; adjust as needed)
TEXT_COLS_DEPLOYABLE = [
    "SAMPLE_SOURCE","METHOD_COLLECTED","GEOLOGIC_AGE","STRATIGRAPHY",
    "HOST_NAME","HOST_ROCK_TYPE","MINERALIZATION","ALTERATION",
    "IGNEOUS_FORM","METAMORPHISM","FACIES_GRADE","LOCATE_DESC",
    "COORDINATES_COMMENT","SAMPLE_COMMENT_ST","SAMPLE_COMMENT_LT","ADDL_ATTR"
]

def add_knowledge_priors(U: gpd.GeoDataFrame, config: dict) -> Tuple[gpd.GeoDataFrame, pd.DataFrame]:
    """Compute embedding-based similarity priors and add them as columns to U."""
    models = load_deposit_models(config['json_dir'])
    embed_fn = get_embedding_client(config)

    model_emb = embed_with_cache(models['model_text'].tolist(), embed_fn, config['embedding_cache'], prefix='model')
    models['emb_dim'] = model_emb.shape[1]

    # Sample text -> embeddings
    text_cols = [c for c in TEXT_COLS_DEPLOYABLE if c in U.columns]
    if not text_cols:
        raise RuntimeError('No text columns found in U for sample embedding. Adjust TEXT_COLS_DEPLOYABLE.')
    sample_texts = [build_sample_text(U.iloc[i], text_cols) for i in range(len(U))]
    sample_emb = embed_with_cache(sample_texts, embed_fn, config['embedding_cache'], prefix='sample')

    # Similarity priors
    sim = cosine_sim_matrix(sample_emb, model_emb)  # (n_samples, n_models)
    # Common, stable priors:
    prior_max = sim.max(axis=1)
    prior_mean_top10 = np.sort(sim, axis=1)[:, -min(10, sim.shape[1]):].mean(axis=1)

    out = U.copy()
    out['prior_sim_max'] = prior_max
    out['prior_sim_top10_mean'] = prior_mean_top10

    # Optional: PCA of model similarities (fold-safe PCA happens in modeling)
    # Keep raw similarities only if you really need them (they can be huge)
    return out, models

## 6) Modeling (spatial CV) + metrics

This block provides:
- baseline model (geochem + spatial features)
- knowledge-augmented model (+ priors)

You can plug in your own target labels.

In [ ]:
def recall_at_top_area(y_true: np.ndarray, y_score: np.ndarray, top_frac: float = 0.10) -> float:
    """Recall among top fraction of highest-scoring samples (proxy for targeting efficiency)."""
    y_true = np.asarray(y_true).astype(int)
    y_score = np.asarray(y_score).astype(float)
    n = len(y_true)
    k = max(1, int(np.ceil(n * top_frac)))
    idx = np.argsort(-y_score)[:k]
    return float(y_true[idx].sum() / max(1, y_true.sum()))

def build_pipeline(feature_cols: List[str], use_pca: bool = False, pca_dim: int = 16) -> Pipeline:
    """Build a robust preprocessing + classifier pipeline."""
    transformers = []
    transformers.append((
        'num',
        Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='median')),
            ('scaler', RobustScaler(with_centering=True)),
            *([('pca', PCA(n_components=pca_dim, random_state=0))] if use_pca else [])
        ]),
        feature_cols
    ))

    pre = ColumnTransformer(transformers=transformers, remainder='drop', verbose_feature_names_out=False)

    clf = HistGradientBoostingClassifier(
        max_depth=6,
        learning_rate=0.05,
        max_iter=600,
        random_state=0
    )
    return Pipeline([('pre', pre), ('clf', clf)])

def spatial_cv_scores(df: pd.DataFrame, y_col: str, group_col: str, feature_cols: List[str], n_splits: int, top_frac: float) -> Dict[str, float]:
    """GroupKFold CV evaluation returning AUC/PR-AUC/recall@top."""
    X = df[feature_cols].copy()
    y = df[y_col].astype(int).values
    groups = df[group_col].values

    cv = GroupKFold(n_splits=n_splits)
    oof = np.zeros(len(df), dtype=float)

    pipe = build_pipeline(feature_cols, use_pca=False)
    for tr, te in cv.split(X, y, groups):
        pipe.fit(X.iloc[tr], y[tr])
        oof[te] = pipe.predict_proba(X.iloc[te])[:, 1]

    auc = roc_auc_score(y, oof) if len(np.unique(y)) > 1 else np.nan
    ap = average_precision_score(y, oof) if len(np.unique(y)) > 1 else np.nan
    rtop = recall_at_top_area(y, oof, top_frac=top_frac)
    return {'auc': float(auc), 'ap': float(ap), f'recall_top_{int(top_frac*100)}pct': float(rtop)}

## 7) Pipeline runner

This is the only cell you should need to run end-to-end (after setting CONFIG).
You still control:
- how you define labels (Cu/Mo thresholds, joint anomaly, etc.)
- which feature columns are used

In [ ]:
def run_pipeline(config: dict) -> Dict[str, object]:
    """Run the core workflow: load -> sites -> geology features -> (optional) knowledge priors."""
    polys, line_layers = load_geology_layers(config['geology_root'], config['metric_crs'])
    gpts, U = load_points_build_sites(config)

    U = add_geology_unit(U, polys)
    U = add_line_distances(U, line_layers, cap_m=config['distance_cap_m'])

    results = {'points': gpts, 'sites': U, 'polys': polys, 'line_layers': line_layers}
    return results

# Example:
# out = run_pipeline(CONFIG)
# U = out['sites']
# U.head()

## 8) Example: define labels + run baseline vs NSAI

Below is a minimal template. Replace the label logic with your study definition.

In [ ]:
# --- Example label definition (placeholder) ---
# You MUST adjust this to match your manuscript definitions.
def define_example_labels(U: gpd.GeoDataFrame) -> gpd.GeoDataFrame:
    """Example only: define a binary label from Cu and/or Mo columns if present."""
    out = U.copy()
    # Try common Cu/Mo names
    cu_col = next((c for c in out.columns if c.lower() in ['cu_ppm','cu','copper_ppm','cu_ppb']), None)
    mo_col = next((c for c in out.columns if c.lower() in ['mo_ppm','mo','molybdenum_ppm','mo_ppb']), None)

    if cu_col is None and mo_col is None:
        raise RuntimeError('Could not find Cu/Mo columns. Please edit label definition.')

    # Simple threshold as an example (not your real science)
    if cu_col is not None:
        cu = pd.to_numeric(out[cu_col], errors='coerce')
    else:
        cu = pd.Series(np.nan, index=out.index)

    if mo_col is not None:
        mo = pd.to_numeric(out[mo_col], errors='coerce')
    else:
        mo = pd.Series(np.nan, index=out.index)

    out['y_anom'] = ((cu > np.nanpercentile(cu, 95)) | (mo > np.nanpercentile(mo, 95))).astype(int)
    return out

# --- Example feature sets ---
def guess_baseline_features(U: gpd.GeoDataFrame) -> List[str]:
    """Heuristic feature selection: geochem numeric columns + distances."""
    dist_cols = [c for c in U.columns if c.startswith('dist_to_')]
    # include common geochem columns if present
    numeric_cols = [c for c in U.columns if pd.api.types.is_numeric_dtype(U[c])]
    # remove target + identifiers
    drop = set(['y_anom'])
    feats = [c for c in numeric_cols if c not in drop]
    # prioritize dist features + numeric; user can edit
    return sorted(set(dist_cols + feats))

# --- Run template ---
# out = run_pipeline(CONFIG)
# U = out['sites']
# U = define_example_labels(U)

# (Optional) add priors
# U, models = add_knowledge_priors(U, CONFIG)

# group column for spatial CV: you should define bins / blocks; here's a simple example using x_bin/y_bin coarsening
# U['cv_block'] = (round_series(U['x_bin'].values, 500.0).astype(int).astype(str) + '_' +
#                  round_series(U['y_bin'].values, 500.0).astype(int).astype(str))

# feats_base = guess_baseline_features(U)
# scores_base = spatial_cv_scores(U, y_col='y_anom', group_col='cv_block', feature_cols=feats_base, n_splits=CONFIG['n_splits'], top_frac=CONFIG['top_frac_eval'])
# print('Baseline:', scores_base)

# feats_nsai = feats_base + ['prior_sim_max','prior_sim_top10_mean']
# scores_nsai = spatial_cv_scores(U, y_col='y_anom', group_col='cv_block', feature_cols=feats_nsai, n_splits=CONFIG['n_splits'], top_frac=CONFIG['top_frac_eval'])
# print('NSAI:', scores_nsai)

## Notes for GitHub

- Put large data (CSV, shapefiles, JSON models) in `data/` **but add to `.gitignore`**.
- Put API keys in environment variables (never in notebook output).
- Commit the refactored notebook + README.